In [37]:
import sympy as smp
from sympy import I, Eq, sin, cos
from IPython.display import display


#common variables
alpha, beta, gamma, sigma, y, dt, t, s, f = smp.symbols('alpha beta gamma sigma y Δt t s f')

# 'y' values at different time steps
yn, yn1, yp, ypp, yppp = smp.symbols('y^n y^{n+1} y\' y\'\' y\'\'\'')

z = alpha + beta * I
m = gamma + sigma * I

#scheme variables
xi, xi1, xi2, xi3, eta = smp.symbols('ξ ξ_1 ξ_2 ξ_3 η')

In [38]:
#list of possible steps ({step_type: equation})
step_lists = {
    "ADV_step" : Eq((yn1-yn)/dt, eta*(f*beta*I*(xi*yn1 + (1-xi)*yn) + s*sigma*I)), #ADV step with ADV source term
    "DIFF_step" : Eq((yn1-yn)/dt, eta*(f*alpha*(xi*yn1 + (1-xi)*yn) + s*gamma)), #DIFF step with DIFF source term
    "ADV_DIFF_step" : Eq((yn1-yn)/dt, eta*(f*z*(xi*yn1 + (1-xi)*yn) + s*m)),
    "ADV_step_fullS" : Eq((yn1-yn)/dt, eta*(f*beta*I*(xi*yn1 + (1-xi)*yn) + s*m)), #ADV step with full source term
    "DIFF_step_fullS" : Eq((yn1-yn)/dt, eta*(f*alpha*(xi*yn1 + (1-xi)*yn) + s*m)), #DIFF step with full source term 
}
#Each step is a tuple of the form [step_type, params], the assemble function depends on the tuple structure to work
#params is a dictionary of the form: {"xi (implicitness)": value, "eta (weight)": value, "s (source term toggle)": value, "f (force term toggle)": value}

#Here is an example recipe for CN, explicit - source - CN, implicit splitting, or (5)(iv) in the document
EXAMPLE_RECIPE = [
    ("ADV_DIFF_step", {"xi": 0, "eta": 0.5, "s": 0, "f": 1.0}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1.0, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 1, "eta": 0.5, "s": 0, "f": 1.0}),
]


def recipe_to_equation(recipe):
    """
    Converts a recipe of steps, input the parameters and turn into a equation.
    recipe: list of tuples
        recipe: list of tuples
            Each tuple contains the step type and a dictionary of parameters.
            Example: [
            ("step_type", 
            {
            "xi (implicitness)": value, 
            "eta (weight)": value, 
            "s (source term toggle)": value, 
            "f (flux term toggle)": value
            }
            )
            ]
    Returns:
        A list of sympy equations Eq(lhs,rhs) representing the steps in the recipe.
    """
    #list of step equations
    steps = []
    for step_type, params in recipe:
        # Substitute the parameters into the step
        eq = step_lists[step_type].subs(xi, params["xi"]).subs(eta, params["eta"]).subs(s, params["s"]).subs(f, params["f"])
        steps.append(eq)

    return steps

#Each step is a tuple of the form (step_type, params), the assemble function depends on the tuple structure to work
step_recipe = [
    ("ADV_DIFF_step", {"xi": 0, "eta": 0.5, "s": 0, "f": 1.0}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1.0, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 1, "eta": 0.5, "s": 0, "f": 1.0}),
]

def assemble_scheme(steps):
    """
    Assembles the equivalent scheme from the list of steps.
    Parameters:
        steps: list of sympy equations
            Each equation is in the form Eq(lhs, rhs) representing the steps in the recipe.
    returns:
        A sympy equation representing the assembled scheme.
    """
    for i in range(len(steps)-1):
        step_sol = smp.solve(steps[i], yn1)[0]
        steps[i+1] = steps[i+1].subs(yn, step_sol)

    return smp.simplify(steps[-1])

def solve_ss(eq, y_n, y_n1):
    """
    Solves for the steady-state solution of a given equation by equating
    y_n to be y_n1.

    Parameters:
        eq: sympy.Equality
            The equation to solve for steady-state in the form 
            Eq(lhs, rhs) with variables yn and yn1

    Returns:
    The simplified steady-state solution of the equation.
    """
    Eq1 = smp.Eq(eq.lhs*dt, eq.rhs*dt)
    Eq2 = Eq1.subs(y_n, y_n1)
    Eq3 = smp.solve(Eq2, y_n1)[0]
    
    return smp.simplify(Eq3)

def scheme_summary(recipe):
    """
    Builds the scheme from recipe and displays the steps, steady state and splitting error from dy/dt = zy.
    Parameters:
        recipe: list of tuples
            Each tuple contains the step type and a dictionary of parameters.
            Example: [
            ("step_type", 
            {
            "xi (implicitness)": value, 
            "eta (weight)": value, 
            "s (source term toggle)": value, 
            "f (flux term toggle)": value
            }
            )
            ]
        analytic: sympy expression
            The analytic solution taylor expansion for dy/dt = zy.
    """
    steps = recipe_to_equation(recipe)
    print("The scheme steps is (In the order of operation):")
    display(*steps)
    assembled_scheme = assemble_scheme(steps)
    steady_state = solve_ss(assembled_scheme, yn, yn1)
    print("Have steady state:")
    display(steady_state)
    print("Have steady state error:")
    display(smp.simplify(smp.expand(steady_state) - smp.expand(-m/z)))

scheme_summary(EXAMPLE_RECIPE)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*y^n*(alpha + I*beta))

Eq((-y^n + y^{n+1})/Δt, 1.0*gamma + 1.0*I*sigma)

Eq((-y^n + y^{n+1})/Δt, 0.5*y^{n+1}*(alpha + I*beta))

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

### The following section showcase splitting schemes branching from the Source-Forcing-Source (4) method and thier steady state errors
Here, we have common step type:
"ADV_DIFF_step", with s=1, f=0
"ADV_DIFF_step" with s=0, f=1
"ADV_DIFF_step" with s=1, f=0

We see this indeed recreate the SFS split from Dubal et al. (2004)

In [39]:
#(4)(i), explicit - source splitting (expect exact steady state)
FS_recipe = [
    ("ADV_DIFF_step", {"xi": 0, "eta": 0, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1, "s": 0, "f": 1}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1, "s": 1, "f": 0}),
]
scheme_summary(FS_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0)

Eq((-y^n + y^{n+1})/Δt, y^n*(alpha + I*beta))

Eq((-y^n + y^{n+1})/Δt, gamma + I*sigma)

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

In [40]:
#(4)(ii), source - implicit splitting (expect exact steady state)
SF_recipe = [
    ("ADV_DIFF_step", {"xi": 1, "eta": 1, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 1, "eta": 1, "s": 0, "f": 1}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 0, "s": 1, "f": 0}),
]
scheme_summary(SF_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, gamma + I*sigma)

Eq((-y^n + y^{n+1})/Δt, y^{n+1}*(alpha + I*beta))

Eq((-y^n + y^{n+1})/Δt, 0)

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

In [41]:
#(4)(iii) Source - Crank Nicolson - Source (expect exact steady state)
SFS_recipe = [
    ("ADV_DIFF_step", {"xi": 0, "eta": 1/2, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 1/2, "eta": 1.0, "s": 0, "f": 1}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1/2, "s": 1, "f": 0}),
]
scheme_summary(SFS_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*gamma + 0.5*I*sigma)

Eq((-y^n + y^{n+1})/Δt, 1.0*(alpha + I*beta)*(0.5*y^n + 0.5*y^{n+1}))

Eq((-y^n + y^{n+1})/Δt, 0.5*gamma + 0.5*I*sigma)

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

### The following section showcase splitting schemes branching from the Forcing-source-forcing (5) method and thier steady state errors
Here, we have common step type:
"ADV_DIFF_step", with s=0, f=1
"ADV_DIFF_step" with s=1, f=0
"ADV_DIFF_step" with s=0, f=1

We see this indeed recreate the FSF split from Dubal et al. (2004)

In [42]:
#(5)(i), explicit-source (expect exact steady state) same as (4)(i)
FS_recipe = [
    ("ADV_DIFF_step", {"xi": 0, "eta": 1, "s": 0, "f": 1}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 0, "s": 0, "f": 1}),
    
]
scheme_summary(FS_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, y^n*(alpha + I*beta))

Eq((-y^n + y^{n+1})/Δt, gamma + I*sigma)

Eq((-y^n + y^{n+1})/Δt, 0)

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

In [43]:
#(5)(ii), source - implicit splitting (expect exact steady state) same as (4)(ii)
SF_recipe = [
    ("ADV_DIFF_step", {"xi": 0, "eta": 0, "s": 0, "f": 1}),
    ("ADV_DIFF_step", {"xi": 1, "eta": 1, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 1, "eta": 1, "s": 0, "f": 1}),
]
scheme_summary(SF_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0)

Eq((-y^n + y^{n+1})/Δt, gamma + I*sigma)

Eq((-y^n + y^{n+1})/Δt, y^{n+1}*(alpha + I*beta))

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

In [44]:
#(5)(iii), CN - source - CN (expect O(Δ𝑡^2) error), different from (4)(iii),
FSF_recipe = [
    ("ADV_DIFF_step", {"xi": 1/2, "eta": 1/2, "s": 0, "f": 1}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 1/2, "eta": 1/2, "s": 0, "f": 1}),

]
scheme_summary(FSF_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*(alpha + I*beta)*(0.5*y^n + 0.5*y^{n+1}))

Eq((-y^n + y^{n+1})/Δt, gamma + I*sigma)

Eq((-y^n + y^{n+1})/Δt, 0.5*(alpha + I*beta)*(0.5*y^n + 0.5*y^{n+1}))

Have steady state:


(0.0625*alpha**2*gamma*Δt**2 + 0.0625*I*alpha**2*sigma*Δt**2 + 0.125*I*alpha*beta*gamma*Δt**2 - 0.125*alpha*beta*sigma*Δt**2 - 0.0625*beta**2*gamma*Δt**2 - 0.0625*I*beta**2*sigma*Δt**2 - 1.0*gamma - 1.0*I*sigma)/(alpha + I*beta)

Have steady state error:


0.0625*Δt**2*(alpha*gamma + I*alpha*sigma + I*beta*gamma - beta*sigma)

In [45]:
#(5)(iv) CN,explicit - source - CN,implicit (expect exact steady state)
FSF_recipe = [
    ("ADV_DIFF_step", {"xi": 0, "eta": 1/2, "s": 0, "f": 1}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 1, "eta": 1/2, "s": 0, "f": 1}),
]
scheme_summary(FSF_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*y^n*(alpha + I*beta))

Eq((-y^n + y^{n+1})/Δt, gamma + I*sigma)

Eq((-y^n + y^{n+1})/Δt, 0.5*y^{n+1}*(alpha + I*beta))

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

In [46]:
#(5)(v) CN,implicit - source - CN,explicit (expect O(Δ𝑡^2) error)
FSF_recipe = [
    ("ADV_DIFF_step", {"xi": 1, "eta": 1/2, "s": 0, "f": 1}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1, "s": 1, "f": 0}),
    ("ADV_DIFF_step", {"xi": 0, "eta": 1/2, "s": 0, "f": 1}),
]
scheme_summary(FSF_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*y^{n+1}*(alpha + I*beta))

Eq((-y^n + y^{n+1})/Δt, gamma + I*sigma)

Eq((-y^n + y^{n+1})/Δt, 0.5*y^n*(alpha + I*beta))

Have steady state:


(0.25*alpha**2*gamma*Δt**2 + 0.25*I*alpha**2*sigma*Δt**2 + 0.5*I*alpha*beta*gamma*Δt**2 - 0.5*alpha*beta*sigma*Δt**2 - 0.25*beta**2*gamma*Δt**2 - 0.25*I*beta**2*sigma*Δt**2 - 1.0*gamma - 1.0*I*sigma)/(alpha + I*beta)

Have steady state error:


0.25*Δt**2*(alpha*gamma + I*alpha*sigma + I*beta*gamma - beta*sigma)

Thus, tested for all the cases mentioned in Dubal et al. (2004), the same conditions holds for a complex $\sigma$ term, where in general, implicit have to be resolved after source to arrive at steady state.

### The following section showcase splitting schemes branching from Advection-Diffusion-Advection splitting and thier steady state errors (with source term resolved on different steps)

In [47]:
#A-D-A CNCS With source only on D

ADA_recipe = [
    ("ADV_step", {"xi": 1/2, "eta": 1/2, "s": 0, "f": 1}),
    ("DIFF_step_fullS", {"xi": 1/2, "eta": 1, "s": 1, "f": 1}),
    ("ADV_step", {"xi": 1/2, "eta": 1/2, "s": 0, "f": 1}),
]
scheme_summary(ADA_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*(0.5*y^n + 0.5*y^{n+1}))

Eq((-y^n + y^{n+1})/Δt, alpha*(0.5*y^n + 0.5*y^{n+1}) + gamma + I*sigma)

Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*(0.5*y^n + 0.5*y^{n+1}))

Have steady state:


(-beta**2*gamma*Δt**2 - I*beta**2*sigma*Δt**2 - 16.0*gamma - 16.0*I*sigma)/(-alpha*beta**2*Δt**2 + 16.0*alpha + 16.0*I*beta)

Have steady state error:


beta**2*Δt**2*(-2.0*alpha*gamma - 2.0*I*alpha*sigma - 1.0*I*beta*gamma + 1.0*beta*sigma)/(-1.0*alpha**2*beta**2*Δt**2 + 16.0*alpha**2 - 1.0*I*alpha*beta**3*Δt**2 + 32.0*I*alpha*beta - 16.0*beta**2)

In [48]:
#A-D-A CNCS With source only on A
ADA_recipe = [
    ("ADV_step_fullS", {"xi": 1/2, "eta": 1/2, "s": 1, "f": 1}),
    ("DIFF_step", {"xi": 1/2, "eta": 1, "s": 0, "f": 1}),
    ("ADV_step_fullS", {"xi": 1/2, "eta": 1/2, "s": 1, "f": 1}),
]
scheme_summary(ADA_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*(0.5*y^n + 0.5*y^{n+1}) + 0.5*gamma + 0.5*I*sigma)

Eq((-y^n + y^{n+1})/Δt, alpha*(0.5*y^n + 0.5*y^{n+1}))

Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*(0.5*y^n + 0.5*y^{n+1}) + 0.5*gamma + 0.5*I*sigma)

Have steady state:


(-2.0*I*alpha*beta*gamma*Δt**2 + 2.0*alpha*beta*sigma*Δt**2 - 16.0*gamma - 16.0*I*sigma)/(-alpha*beta**2*Δt**2 + 16.0*alpha + 16.0*I*beta)

Have steady state error:


alpha*beta*Δt**2*(-2.0*I*alpha*gamma + 2.0*alpha*sigma + 1.0*beta*gamma + 1.0*I*beta*sigma)/(-1.0*alpha**2*beta**2*Δt**2 + 16.0*alpha**2 - 1.0*I*alpha*beta**3*Δt**2 + 32.0*I*alpha*beta - 16.0*beta**2)

In [49]:
#A-D-A CNCS With source on all steps
ADA_recipe = [
    ("ADV_step", {"xi": 1/2, "eta": 1/2, "s": 1, "f": 1}),
    ("DIFF_step", {"xi": 1/2, "eta": 1, "s": 1, "f": 1}),
    ("ADV_step", {"xi": 1/2, "eta": 1/2, "s": 1, "f": 1}),
]
scheme_summary(ADA_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*(0.5*y^n + 0.5*y^{n+1}) + 0.5*I*sigma)

Eq((-y^n + y^{n+1})/Δt, alpha*(0.5*y^n + 0.5*y^{n+1}) + gamma)

Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*(0.5*y^n + 0.5*y^{n+1}) + 0.5*I*sigma)

Have steady state:


(2.0*alpha*beta*sigma*Δt**2 - beta**2*gamma*Δt**2 - 16.0*gamma - 16.0*I*sigma)/(-alpha*beta**2*Δt**2 + 16.0*alpha + 16.0*I*beta)

Have steady state error:


((alpha + I*beta)*(2.0*alpha*beta*sigma*Δt**2 - beta**2*gamma*Δt**2 - 16.0*gamma - 16.0*I*sigma) + (gamma + I*sigma)*(-alpha*beta**2*Δt**2 + 16.0*alpha + 16.0*I*beta))/((alpha + I*beta)*(-alpha*beta**2*Δt**2 + 16.0*alpha + 16.0*I*beta))

In [50]:
#A-D-A FTCS With source on all steps
ADA_recipe = [
    ("ADV_step", {"xi": 0, "eta": 1/2, "s": 1, "f": 1}),
    ("DIFF_step", {"xi": 0, "eta": 1, "s": 1, "f": 1}),
    ("ADV_step", {"xi": 0, "eta": 1/2, "s": 1, "f": 1}),
]
scheme_summary(ADA_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*y^n + 0.5*I*sigma)

Eq((-y^n + y^{n+1})/Δt, alpha*y^n + gamma)

Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*y^n + 0.5*I*sigma)

Have steady state:


(alpha*beta*sigma*Δt**2 - 2.0*I*alpha*sigma*Δt - 2.0*I*beta*gamma*Δt + beta*sigma*Δt - 4.0*gamma - 4.0*I*sigma)/(-alpha*beta**2*Δt**2 + 4.0*I*alpha*beta*Δt + 4.0*alpha - beta**2*Δt + 4.0*I*beta)

Have steady state error:


((alpha + I*beta)*(alpha*beta*sigma*Δt**2 - 2.0*I*alpha*sigma*Δt - 2.0*I*beta*gamma*Δt + beta*sigma*Δt - 4.0*gamma - 4.0*I*sigma) + (gamma + I*sigma)*(-alpha*beta**2*Δt**2 + 4.0*I*alpha*beta*Δt + 4.0*alpha - beta**2*Δt + 4.0*I*beta))/((alpha + I*beta)*(-alpha*beta**2*Δt**2 + 4.0*I*alpha*beta*Δt + 4.0*alpha - beta**2*Δt + 4.0*I*beta))

In [51]:
#A-D-A CN-E CN CN-I With source only on D

ADA_recipe = [
    ("ADV_step", {"xi": 0, "eta": 1/2, "s": 0, "f": 1}),
    ("DIFF_step_fullS", {"xi": 1/2, "eta": 1, "s": 1, "f": 1}),
    ("ADV_step", {"xi": 1, "eta": 1/2, "s": 0, "f": 1}),
]
scheme_summary(ADA_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*y^n)

Eq((-y^n + y^{n+1})/Δt, alpha*(0.5*y^n + 0.5*y^{n+1}) + gamma + I*sigma)

Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*y^{n+1})

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

In [52]:
#A-D-A CN-E CN CN-I With source only on A

ADA_recipe = [
    ("ADV_step_fullS", {"xi": 0, "eta": 1/2, "s": 1, "f": 1}),
    ("DIFF_step", {"xi": 1/2, "eta": 1, "s": 0, "f": 1}),
    ("ADV_step_fullS", {"xi": 1, "eta": 1/2, "s": 1, "f": 1}),
]
scheme_summary(ADA_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*y^n + 0.5*gamma + 0.5*I*sigma)

Eq((-y^n + y^{n+1})/Δt, alpha*(0.5*y^n + 0.5*y^{n+1}))

Eq((-y^n + y^{n+1})/Δt, 0.5*I*beta*y^{n+1} + 0.5*gamma + 0.5*I*sigma)

Have steady state:


(-gamma - I*sigma)/(alpha + I*beta)

Have steady state error:


0

In [53]:
#BTCS DA:
BTCS_recipe = [
    ("DIFF_step", {"xi": 1, "eta": 1, "s": 0, "f": 1}),
    ("ADV_step", {"xi": 1, "eta": 1, "s": 0, "f": 1}),
]
scheme_summary(BTCS_recipe)

The scheme steps is (In the order of operation):


Eq((-y^n + y^{n+1})/Δt, alpha*y^{n+1})

Eq((-y^n + y^{n+1})/Δt, I*beta*y^{n+1})

Have steady state:


0

Have steady state error:


(gamma + I*sigma)/(alpha + I*beta)